In [1]:
from collections import Counter
from itertools import combinations
from math import sqrt
import os
# os.environ["CUDA_VISIBLE_DEVICES"] = "0" #comment out this line if you want to use gpu
import random
from keras.layers import Concatenate, Dense, Dot, Dropout, Embedding, Input, Reshape
from keras.models import Model
from keras.callbacks import Callback, ModelCheckpoint
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
import tensorflow

In [2]:
import random
random.seed(2021)
np.random.seed(2021)
# tf > 2.0
tensorflow.random.set_seed(2021)
#tf < 2.0
#tf.set_random_seed(2021)

In [3]:
'''
params:
    -pred: an array containing all predicted ratings
    -actual: an array containing all ground truth ratings
    
return:
    a scalar whose value is the rmse
'''
def rmse(pred, actual):
    # Ignore ratings with value zero.
    pred = pred[actual.nonzero()].flatten()
    actual = actual[actual.nonzero()].flatten()
    return sqrt(mean_squared_error(pred, actual))

In [4]:
'''
params:
    -len_continuous: number of continuous features
    -deep_vocab_lens: an array of integers where deep_vocab_lens[i] represents the number of unique values of (i+1)-th deep 
        categorical feature
    -len_wide: number of wide features
    -embed_size: dimension of the embedding vectors of deep categorical features
    
return:
    a keras Model object for the constructed wdl model 
'''


def build_wdl_model(len_continuous, deep_vocab_lens, len_wide, embed_size):
    # A list containing all input layers
    input_list = []
    
    # Input layer for continuous features
    continuous_input = Input(shape=(len_continuous,), dtype='float32', name='continuous_input')
    input_list.append(continuous_input)
    
    
    # Get embeddings for all deep categorical features
    emb_list = []
    for vocab_size in deep_vocab_lens:
        _input = Input(shape=(1,), dtype='int32')
        input_list.append(_input)
        _emb = Embedding(output_dim=embed_size, input_dim=vocab_size, input_length=1)(_input)
        _emb = Reshape((embed_size,))(_emb)
        emb_list.append(_emb)
    
    
   
    # Create input layer for deep component by concatenating the embeddings and continuous features' input layer
    deep_input = Concatenate()(emb_list + [continuous_input])
    

    # Construct deep component
    dense_1 = Dense(256, activation='relu')(deep_input)
    dense_1_dp = Dropout(0.3)(dense_1)
    dense_2 = Dense(128, activation='relu')(dense_1_dp)
    dense_2_dp = Dropout(0.3)(dense_2)
    dense_3 = Dense(64, activation='relu')(dense_2_dp)
    dense_3_dp = Dropout(0.3)(dense_3)

    
    # Create input layer for wide component
    wide_input = Input(shape=(len_wide,), dtype='float32')
    input_list.append(wide_input)

    
    # Concatenate the outputs of deep and wide components and feed the 
    # concatenated vector into the finall fully connected layer
    fc_input = Concatenate()([dense_3_dp, wide_input])
    model_output = Dense(1)(fc_input)
    
    model = Model(inputs=input_list,
                  outputs=model_output)
    return model
    

In [5]:
'''
params:
    -df: input dataframe
    -continuous_columns: column names of continuous features
    
return: 
    a numpy array where each row contains the values of continuous features in the corresponding row of the
    input dataframe
'''
def get_continuous_features(df, continuous_columns):
    continuous_features = df[continuous_columns].values
    return continuous_features

In [6]:
'''
params:
    -df: input dataframe
    -comb_p: number of elements in each combination (e.g., there are two elements in the combination {fried chicken, chicken and 
    waffle}, and three elements in the combination {fried chicken, chicken and waffle, chicken fried rice})
    -topk: number of mostly frequent combinations to retrieve
    -output_freq: whether to return the frequencies of retrieved combinations
    
return:
    1. output_freq = True: a list X where each element is a tuple containing a combinantion tuple and corresponding frequency, and the 
        elements are stored in the descending order of their frequencies
    2. output_freq = False: a list X where each element is a tuple containing a combinantion tuple, and the elements are stored in 
    the descending order of their frequencies
'''
def get_top_k_p_combinations(df, comb_p, topk, output_freq=False):
    # get all combinations with comb_p
    def get_category_combinations(categories_str, comb_p=2):
        categories = categories_str.split(', ')
        return list(combinations(categories, comb_p))
    # [('Lounges', 'Dance Clubs'), ('Lounges', 'Bars'), ('Lounges', 'Nightlife'), ('Dance Clubs', 'Bars'), ('Dance Clubs', 'Nightlife'), ('Bars', 'Nightlife')]
    all_categories_p_combos = df["item_categories"].apply(
        lambda x: get_category_combinations(x, comb_p)).values.tolist()
    # ('Lounges', 'Dance Clubs')
    # list of tuples that each index refer to one combination
    all_categories_p_combos = [tuple(t) for item in all_categories_p_combos for t in item]

    tmp = dict(Counter(all_categories_p_combos))
    sorted_categories_combinations = list(sorted(tmp.items(), key=lambda x: x[1], reverse=True))
    if output_freq:
        return sorted_categories_combinations[:topk]
    else:
        return [t[0] for t in sorted_categories_combinations[:topk]]

In [7]:
'''
params:
    -df: input dataframe
    -selected_categories_to_idx: a dictionary mapping item categories to corrresponding integral indices
    -top_combinations: a list containing retrieved mostly frequent combinantions of item categories
    
return:
    a numpy array where each row contains the categorical features' binary encodings and cross product
    transformations for the corresponding row of the input dataframe
'''

def get_wide_features(df, selected_categories_to_idx, top_combinations):
    def categories_to_binary_output(categories):
        binary_output = [0 for _ in range(len(selected_categories_to_idx))]
        for category in categories.split(', '):
            if category in selected_categories_to_idx:
                binary_output[selected_categories_to_idx[category]] = 1
            else:
                binary_output[0] = 1
        return binary_output
    def categories_cross_transformation(categories):
        current_category_set = set(categories.split(', '))
        corss_transform_output = [0 for _ in range(len(top_combinations))]
        for k, comb_k in enumerate(top_combinations):
            if len(current_category_set & comb_k) == len(comb_k):
                corss_transform_output[k] = 1
            else:
                corss_transform_output[k] = 0
        return corss_transform_output

    category_binary_features = np.array(df.item_categories.apply(
        lambda x: categories_to_binary_output(x)).values.tolist())
    print('category_binary_features shape:',category_binary_features.shape)
    category_corss_transform_features = np.array(df.item_categories.apply(
        lambda x: categories_cross_transformation(x)).values.tolist())
    print('category_cross_features shape:',category_corss_transform_features.shape)
    out = np.concatenate((category_binary_features, category_corss_transform_features), axis=1)
    print('wide features shape:',out.shape)
    return np.concatenate((category_binary_features, category_corss_transform_features), axis=1)


In [8]:
tr_df = pd.read_csv("data/train.csv")
val_df = pd.read_csv("data/valid.csv")
te_df = pd.read_csv("data/test.csv")

tr_ratings = tr_df.stars.values
val_ratings = val_df.stars.values



In [9]:
user_df = pd.read_csv("data/user.csv")
item_df = pd.read_csv("data/business.csv")

# Rename some columns of dfs and convert the indices of dfs into string type for easier reference in later stage 
user_df = user_df.rename(index=str, columns={t: 'user_' + t for t in user_df.columns if t != 'user_id'})
item_df = item_df.rename(index=str, columns={t: 'item_' + t for t in item_df.columns if t != 'business_id'})


In [10]:
# Save the original row indices of each rating table
tr_df["index"] = tr_df.index
val_df["index"]  = val_df.index
te_df["index"] = te_df.index

tr_df = pd.merge(pd.merge(tr_df, user_df, on='user_id'), item_df, on='business_id').sort_values(by=['index']).reset_index(drop=True)
val_df = pd.merge(pd.merge(val_df, user_df, on='user_id'), item_df, on='business_id').sort_values(by=['index']).reset_index(drop=True)
te_df = pd.merge(pd.merge(te_df, user_df, on='user_id'), item_df, on='business_id').sort_values(by=['index']).reset_index(drop=True)



In [11]:
# Specify the columns containing conitnuous features
continuous_columns = ["user_average_stars", "user_cool", "user_fans", 
                      "user_review_count", "user_useful", "user_funny",
                      "item_is_open", "item_latitude", "item_longitude", 
                      "item_review_count", "item_stars"]

# Get values of continous features for train/validation/test sets using the utility function defined previously

tr_continuous_features = get_continuous_features(tr_df, continuous_columns)
val_continuous_features = get_continuous_features(val_df, continuous_columns)
te_continuous_features = get_continuous_features(te_df, continuous_columns)

# Standardize each feature by removing the mean of the training samples and scaling to unit variance.
# See https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html for more details.
scaler = StandardScaler().fit(tr_continuous_features)

tr_continuous_features = scaler.transform(tr_continuous_features)
val_continuous_features = scaler.transform(val_continuous_features)
te_continuous_features = scaler.transform(te_continuous_features)

In [12]:
# Sepcify column names of deep categorical features
item_deep_columns = ["item_city", "item_postal_code", "item_state"]

# An array of integers where deep_vocab_lens[i] represents the number of unique values of (i+1)-th deep categorical feature
item_deep_vocab_lens = []

for col_name in item_deep_columns:
    # Get all unique values of this deep categorical feature
    tmp = item_df[col_name].unique()
    
    # Create a dictionary mapping each unique value to a unique integral index
    vocab = dict(zip(tmp, range(1, len(tmp) + 1)))
    
    # Get the number of unique values of this deep categorical features
    item_deep_vocab_lens.append(len(vocab) + 1)
    
    # Create a new column where each entry stores the integral index of this deep categorical feature's value in the same row
    item_df[col_name + "_idx"] = item_df[col_name].apply(lambda x: vocab[x])


# Create a dictionary mapping each business id to corresponding values of deep categorical features
item_deep_idx_columns = [t + "_idx" for t in item_deep_columns]
item_to_deep_categorical_features = dict(zip(item_df.business_id.values, item_df[item_deep_idx_columns].values.tolist()))

# Creat numpy arrays storing corresponding deep categorical features' values of train/validation/test sets using the above mapping
tr_deep_categorical_features = np.array(tr_df.business_id.apply(lambda x: item_to_deep_categorical_features[x]).values.tolist())
val_deep_categorical_features = np.array(val_df.business_id.apply(lambda x: item_to_deep_categorical_features[x]).values.tolist())
te_deep_categorical_features = np.array(te_df.business_id.apply(lambda x: item_to_deep_categorical_features[x]).values.tolist())


In [13]:
# Collect the categories of all items 
all_categories = [category for category_list in item_df.item_categories.values for category in category_list.split(", ")]

# Sort all unique values of the item categories by their frequencies in descending order
category_sorted = sorted(Counter(all_categories).items(), key=lambda x: x[1], reverse=True)

# Select top 500 most frequent categories
selected_categories = [t[0] for t in category_sorted[:500]]

# Create a dictionary mapping each secleted category to a unique integral index
selected_categories_to_idx = dict(zip(selected_categories, range(1, len(selected_categories) + 1)))

# Map all categories unseen in the item df to index 0
selected_categories_to_idx['unk'] = 0

# Create a dictionary mapping each integral index to corresponding category
idx_to_selected_categories = {val: key for key, val in selected_categories_to_idx.items()}


In [14]:
# Get most frequent categories combinantions using the utility function defined previously and store them in the folloing list
top_combinations = []

# Get top 50 most frequent two-categories combinantions in the train set

top_combinations += get_top_k_p_combinations(tr_df, 2, 50, output_freq=False)

# Get top 30 most frequent three-categories combinantions in the train set
top_combinations += get_top_k_p_combinations(tr_df, 3, 30, output_freq=False)

# Get top 20 most frequent four-categories combinantions in the train set
top_combinations += get_top_k_p_combinations(tr_df, 4, 20, output_freq=False)

# Convert each combinantion in the list to a set data structure
top_combinations = [set(t) for t in top_combinations]

In [15]:
# Get values of wide features for train/validation/test sets using the utility function defined previously

tr_wide_features = get_wide_features(tr_df, selected_categories_to_idx, top_combinations)
val_wide_features = get_wide_features(val_df, selected_categories_to_idx, top_combinations)
te_wide_features = get_wide_features(te_df, selected_categories_to_idx, top_combinations)


category_binary_features shape: (60080, 501)
category_cross_features shape: (60080, 100)
wide features shape: (60080, 601)
category_binary_features shape: (7510, 501)
category_cross_features shape: (7510, 100)
wide features shape: (7510, 601)
category_binary_features shape: (7510, 501)
category_cross_features shape: (7510, 100)
wide features shape: (7510, 601)


In [16]:
tr_wide_features

array([[0, 1, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0]])

In [17]:
tr_features = [tr_continuous_features,tr_deep_categorical_features[:,0],tr_deep_categorical_features[:,1],tr_deep_categorical_features[:,2],tr_wide_features]
# tr_features = []
# tr_features.append(tr_continuous_features.tolist())
# tr_features += [tr_deep_categorical_features[:,i].tolist() for i in range(tr_deep_categorical_features.shape[1])]
# tr_features.append(tr_wide_features.tolist())

val_features = [val_continuous_features,val_deep_categorical_features[:,0],val_deep_categorical_features[:,1],val_deep_categorical_features[:,2],val_wide_features]
# val_features = []
# val_features.append(val_continuous_features.tolist())
# val_features += [val_deep_categorical_features[:,i].tolist() for i in range(val_deep_categorical_features.shape[1])]
# val_features.append(val_wide_features.tolist())

te_features = []
te_features.append(te_continuous_features.tolist())
te_features += [te_deep_categorical_features[:,i].tolist() for i in range(te_deep_categorical_features.shape[1])]
te_features.append(te_wide_features.tolist())

In [18]:
tr_features

[array([[-1.3106592 , -0.29126561, -0.29606097, ..., -0.62640436,
         -0.32860267,  0.27585547],
        [ 1.8933461 ,  0.27159911,  0.3575088 , ..., -0.38901381,
         -0.44103617,  0.27585547],
        [ 0.5683815 , -0.26975485, -0.16718806, ..., -0.62240987,
         -0.31827715, -0.69945181],
        ...,
        [-0.22659725, -0.28051023, -0.15798285, ...,  1.91534585,
         -0.59706634, -0.69945181],
        [-1.11793708, -0.08998641,  0.08135256, ...,  1.92068597,
         -0.59821362, -1.6747591 ],
        [-0.73249283, -0.3097034 , -0.30526618, ...,  1.87761793,
         -0.20469636,  1.25116276]]),
 array([ 8, 21,  8, ...,  6, 10,  1]),
 array([ 86, 193,   8, ..., 470, 655, 189]),
 array([3, 2, 3, ..., 5, 5, 1]),
 array([[0, 1, 0, ..., 0, 0, 0],
        [0, 1, 0, ..., 0, 0, 0],
        [0, 1, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 1, 0, ..., 0, 0, 0],
        [0, 1, 0, ..., 0, 0, 0]])]

In [19]:
wdl_model = build_wdl_model(
        len(tr_continuous_features[0]),
        item_deep_vocab_lens,   # num of category classes
        len(tr_wide_features[0]), 
        embed_size=100)
#print(len(tr_continuous_features[0]))
#print(item_deep_vocab_lens)
#print(len(tr_wide_features[0]))

In [20]:
wdl_model.compile(optimizer='adam', loss='mse')

history = wdl_model.fit(
        tr_features, 
        tr_ratings, 
        epochs=5, verbose=1, callbacks=[ModelCheckpoint('model.h5')])

Epoch 1/5
1878/1878 [==============================] - 4s 2ms/step - loss: 2.1569
Epoch 2/5
1878/1878 [==============================] - 4s 2ms/step - loss: 1.1968
Epoch 3/5
1878/1878 [==============================] - 4s 2ms/step - loss: 1.0966
Epoch 4/5
1878/1878 [==============================] - 4s 2ms/step - loss: 1.0738
Epoch 5/5
1878/1878 [==============================] - 4s 2ms/step - loss: 1.0669


In [21]:
y_pred = wdl_model.predict(tr_features)
print("TRAIN RMSE: ", rmse(y_pred, tr_ratings))
y_pred = wdl_model.predict(val_features)
print("VALID RMSE: ", rmse(y_pred, val_ratings))

TRAIN RMSE:  0.9945750224254961
VALID RMSE:  1.0047115652332168


In [26]:
Train_RMSE = []
Valid_RMSE = []
train_optimizer = []
train_epoch = []
train_embed_size = []

optimizer = ['adagrad', 'adam']
epoch =  [3,5,7,9,10,15]
embed_size = [25,50,100,150,200]

for o in optimizer:
    for e in epoch:
        for es in embed_size:
            wdl_model = build_wdl_model(
            len(tr_continuous_features[0]),
            item_deep_vocab_lens,   # num of category classes
            len(tr_wide_features[0]), 
            embed_size=es)
            
            wdl_model.compile(optimizer=o, loss='mse')

            history = wdl_model.fit(
                tr_features, 
                tr_ratings, 
                epochs=e, verbose=1, callbacks=[ModelCheckpoint('model.h5')])
            
            print("optimizer",o)
            print("epoch",e)
            print("embed_size",es)
            y_pred = wdl_model.predict(tr_features)
            print("TRAIN RMSE: ", rmse(y_pred, tr_ratings))
            Train_RMSE.append(rmse(y_pred, tr_ratings))
            y_pred = wdl_model.predict(val_features)
            print("VALID RMSE: ", rmse(y_pred, val_ratings))
            train_optimizer.append(o)
            train_epoch.append(e)
            train_embed_size.append(es)
            Valid_RMSE.append(rmse(y_pred, val_ratings))
            

Epoch 1/3
1878/1878 [==============================] - 3s 1ms/step - loss: 5.7990
Epoch 2/3
1878/1878 [==============================] - 2s 1ms/step - loss: 1.7612
Epoch 3/3
1878/1878 [==============================] - 2s 1ms/step - loss: 1.6048
optimizer adagrad
epoch 3
embed_size 25
TRAIN RMSE:  1.0822131114990294
VALID RMSE:  1.0670337077548389
Epoch 1/3
1878/1878 [==============================] - 3s 1ms/step - loss: 5.6780
Epoch 2/3
1878/1878 [==============================] - 2s 1ms/step - loss: 1.6819
Epoch 3/3
1878/1878 [==============================] - 2s 1ms/step - loss: 1.5507
optimizer adagrad
epoch 3
embed_size 50
TRAIN RMSE:  1.0671137363765368
VALID RMSE:  1.051709342225936
Epoch 1/3
1878/1878 [==============================] - 4s 2ms/step - loss: 6.2486
Epoch 2/3
1878/1878 [==============================] - 3s 2ms/step - loss: 1.6603
Epoch 3/3
1878/1878 [==============================] - 3s 2ms/step - loss: 1.5604
optimizer adagrad
epoch 3
embed_size 100
TRAIN RMSE:  1

1878/1878 [==============================] - 3s 2ms/step - loss: 1.4610
Epoch 7/9
1878/1878 [==============================] - 2s 1ms/step - loss: 1.4239
Epoch 8/9
1878/1878 [==============================] - 2s 1ms/step - loss: 1.4140
Epoch 9/9
1878/1878 [==============================] - 2s 1ms/step - loss: 1.3961
optimizer adagrad
epoch 9
embed_size 25
TRAIN RMSE:  1.0394232075645562
VALID RMSE:  1.026867327494554
Epoch 1/9
1878/1878 [==============================] - 3s 1ms/step - loss: 5.2127
Epoch 2/9
1878/1878 [==============================] - 3s 1ms/step - loss: 1.7450
Epoch 3/9
1878/1878 [==============================] - 3s 2ms/step - loss: 1.5900
Epoch 4/9
1878/1878 [==============================] - 3s 1ms/step - loss: 1.5279
Epoch 5/9
1878/1878 [==============================] - 3s 1ms/step - loss: 1.4885
Epoch 6/9
1878/1878 [==============================] - 3s 1ms/step - loss: 1.4548
Epoch 7/9
1878/1878 [==============================] - 3s 1ms/step - loss: 1.4359
Epoch

Epoch 9/10
1878/1878 [==============================] - 5s 2ms/step - loss: 1.3610
Epoch 10/10
1878/1878 [==============================] - 5s 2ms/step - loss: 1.3646
optimizer adagrad
epoch 10
embed_size 200
TRAIN RMSE:  1.0228028159797045
VALID RMSE:  1.013369643116387
Epoch 1/15
1878/1878 [==============================] - 4s 2ms/step - loss: 6.5950
Epoch 2/15
1878/1878 [==============================] - 4s 2ms/step - loss: 1.8145
Epoch 3/15
1878/1878 [==============================] - 4s 2ms/step - loss: 1.6457
Epoch 4/15
1878/1878 [==============================] - 4s 2ms/step - loss: 1.5629
Epoch 5/15
1878/1878 [==============================] - 4s 2ms/step - loss: 1.5341
Epoch 6/15
1878/1878 [==============================] - 4s 2ms/step - loss: 1.5003
Epoch 7/15
1878/1878 [==============================] - 4s 2ms/step - loss: 1.4607
Epoch 8/15
1878/1878 [==============================] - 4s 2ms/step - loss: 1.4405
Epoch 9/15
1878/1878 [==============================] - 4s 2ms/s

1878/1878 [==============================] - 6s 3ms/step - loss: 1.2115
Epoch 3/3
1878/1878 [==============================] - 4s 2ms/step - loss: 1.1097
optimizer adam
epoch 3
embed_size 150
TRAIN RMSE:  1.0037064616820897
VALID RMSE:  1.0084302007691774
Epoch 1/3
1878/1878 [==============================] - 6s 3ms/step - loss: 2.0754
Epoch 2/3
1878/1878 [==============================] - 6s 3ms/step - loss: 1.2334
Epoch 3/3
1878/1878 [==============================] - 5s 3ms/step - loss: 1.1191
optimizer adam
epoch 3
embed_size 200
TRAIN RMSE:  1.0066459483661403
VALID RMSE:  1.013600004010235
Epoch 1/5
1878/1878 [==============================] - 3s 1ms/step - loss: 2.2144
Epoch 2/5
1878/1878 [==============================] - 3s 1ms/step - loss: 1.2112
Epoch 3/5
1878/1878 [==============================] - 3s 1ms/step - loss: 1.1069
Epoch 4/5
1878/1878 [==============================] - 3s 2ms/step - loss: 1.0785
Epoch 5/5
1878/1878 [==============================] - 3s 2ms/step - 

Epoch 1/9
1878/1878 [==============================] - 5s 2ms/step - loss: 2.1313
Epoch 2/9
1878/1878 [==============================] - 4s 2ms/step - loss: 1.2167
Epoch 3/9
1878/1878 [==============================] - 4s 2ms/step - loss: 1.1100
Epoch 4/9
1878/1878 [==============================] - 4s 2ms/step - loss: 1.0823
Epoch 5/9
1878/1878 [==============================] - 4s 2ms/step - loss: 1.0641
Epoch 6/9
1878/1878 [==============================] - 5s 2ms/step - loss: 1.0376
Epoch 7/9
1878/1878 [==============================] - 4s 2ms/step - loss: 1.0293
Epoch 8/9
1878/1878 [==============================] - 4s 2ms/step - loss: 1.0156
Epoch 9/9
1878/1878 [==============================] - 4s 2ms/step - loss: 1.0192
optimizer adam
epoch 9
embed_size 100
TRAIN RMSE:  0.9850214766891505
VALID RMSE:  1.00707474022831
Epoch 1/9
1878/1878 [==============================] - 5s 2ms/step - loss: 1.9705
Epoch 2/9
1878/1878 [==============================] - 4s 2ms/step - loss: 1.202

1878/1878 [==============================] - 3s 2ms/step - loss: 0.9987
Epoch 14/15
1878/1878 [==============================] - 3s 1ms/step - loss: 1.0041
Epoch 15/15
1878/1878 [==============================] - 3s 1ms/step - loss: 0.9874
optimizer adam
epoch 15
embed_size 25
TRAIN RMSE:  0.9779276838599867
VALID RMSE:  1.0103750483881286
Epoch 1/15
1878/1878 [==============================] - 4s 2ms/step - loss: 2.2238
Epoch 2/15
1878/1878 [==============================] - 3s 2ms/step - loss: 1.2172
Epoch 3/15
1878/1878 [==============================] - 4s 2ms/step - loss: 1.1172
Epoch 4/15
1878/1878 [==============================] - 4s 2ms/step - loss: 1.0848
Epoch 5/15
1878/1878 [==============================] - 3s 2ms/step - loss: 1.0654
Epoch 6/15
1878/1878 [==============================] - 3s 2ms/step - loss: 1.0394
Epoch 7/15
1878/1878 [==============================] - 3s 2ms/step - loss: 1.0273
Epoch 8/15
1878/1878 [==============================] - 3s 2ms/step - loss: 1

In [27]:
result = pd.DataFrame()
   
result['optimizer'] = train_optimizer
result['epoch '] = train_epoch 
result['embed_size'] = train_embed_size
result['Train_RMSE'] = Train_RMSE
result['Valid_RMSE'] = Valid_RMSE


In [31]:
result = result.sort_values(by=['Valid_RMSE'], ascending=True)

In [32]:
result

,optimizer,epoch,embed_size,Train_RMSE,Valid_RMSE
48,adam,9,150,0.987072,1.004304
40,adam,7,25,0.992064,1.004400
46,adam,9,50,0.984903,1.004768
44,adam,7,200,0.994542,1.005023
35,adam,5,25,0.994214,1.005433
52,adam,10,100,0.987192,1.005979
39,adam,5,200,0.996826,1.006129
37,adam,5,100,0.994725,1.006320
41,adam,7,50,0.991031,1.006323
43,adam,7,150,0.993212,1.006612
